In [1]:
import os
import warnings
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
warnings.filterwarnings("ignore")
import logging

## 读取数据

In [2]:
print("read data")
path = "/home/zfwang/Downloads/data/aichallenge_2019_ad_fraud/data/"
df_train = pd.read_csv(path + "round1_iflyad_anticheat_traindata.txt", sep = "\t")
df_test = pd.read_csv(path + "round1_iflyad_anticheat_testdata_feature.txt", sep = "\t")
df_uni = pd.concat([df_train, df_test], ignore_index = True)
df_uni["label"] = df_uni["label"].fillna(-1).astype(int)

read data


In [3]:
with pd.option_context("display.max_columns", None):
    print(df_train.head())
    print("=" * 100)
    print(df_uni.head())

                                                 sid  label  \
0  d7460126-e071-4979-9ee8-42f72777a28a-156009070...      1   
1  b660d559-db97-4b5f-9bd2-2450cb89ce77-156005074...      1   
2  f49a740e-66c3-4605-9b67-4d3079fe69cb-156008914...      0   
3  fd60d096-f168-4540-b782-729d64d0fcc6-156006253...      0   
4  a037b032-a5c7-40ea-9161-26b118b12406-156007938...      1   

                            pkgname              ver  \
0  2d2664e827bcbb8b07100c7fbe072e9b          11.11.0   
1                             empty              NaN   
2  16b81f93f4b1a35cebbf15f07683f171  3.2.1.0524.1958   
3                             empty            4.7.5   
4  cf4821986014a7fef9d7b7ad8de655e4              228   

                       adunitshowid                       mediashowid  \
0  907d0f8c29663840491577a21c7b612a  ca64a500000d84c8fcb8a0587d0e1e0c   
1  10199dc8fea2e173525bc3151bd84312  3f2fc57a6e1f9c6fa4464c25cc1e88a3   
2  83f2ecfe65f936f5f2ed59f8e8ff1d01  eea7280e1a2313e4c2e89290b01d

## 类别型特征和无用特征

In [4]:
cat_cols = ['pkgname', 'ver', 'adunitshowid', 'mediashowid', 'apptype', 'ip',
           'reqrealip', 'city', 'province', 'adidmd5', 'imeimd5', 'idfamd5',
           'openudidmd5', 'macmd5', 'dvctype', 'model', 'make', 'ntt',
           'carrier', 'os', 'osv', 'orientation', 'lan', 'h', 'w', 'ppi']
drop_cols = ["sid", 'label', 'nginxtime']

## 缺失值填充

In [5]:
print("full null")
for cat_col in cat_cols:
    if df_uni[cat_col].isnull().sum() > 0:
        df_uni[cat_col].fillna("null_value", inplace = True)

full null


In [ ]:
def gen_value_counts(data, col):
    print('value counts', col)
    df_tmp = pd.DataFrame(data[col].value_counts().reset_index())
    df_tmp.columns = [col, 'tmp']
    r = pd.merge(data, df_tmp, how='left', on=col)['tmp']
    return r.fillna(0)

value_counts_col = ["pkgname", "adunitshowid", "ip", "reqrealip",
                    "adidmd5", "imeimd5", "idfamd5", "macmd5"]

for col in value_counts_col:
    df_uni["vc_" + col] = gen_value_counts(df_uni, col)

In [ ]:
print("cut")
def cut_col(data, col_name, cut_list):
    print("cutting", "col_name")
    
    def _trans(array):
        count = array(["box_counts"])
        for box in cut_list:
            if count <= box:
                return "count_" + str(box)
        return array[col_name]

    df_counts = pd.DataFrame(data[col_name].value_counts())
    df_counts.columns = ["box_counts"]
    df_counts[col_name] = df_counts.index
    df = pd.merge(data, df_counts, on = col_name, how = "left")
    column =  df.apply(_trans, axis = 1)
    return column

cut_col_dict = {
    ('pkgname', 'ver', 'reqrealip', 'adidmd5', 'imeimd5', 
     'openudidmd5', 'macmd5', 'model', 'make'): [3],
    ('ip',): [3, 5, 10],
}

for cut_cols, cut_list in cut_col_dict.items():
    for col in cut_cols:
        df_uni[col] = cut_col(df_uni, col, cut_list)